### 신경망 모델 구성

- 신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성. 
- torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공. PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass)임. 
- 신경망은 다른 모듈(계층; layer)로 구성된 모듈. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리 가능.

In [5]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#### 학습을 위한 장치 구성

In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


#### 클래스 정의

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

- NeuralNetwork 의 인스턴스(instance)를 생성, 이를 device 로 이동한 뒤, 구조(structure) 출력
- model.forward() 를 직접 호출하지 말것!

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


- 모델에 입력을 전달하여 호출하면 2차원 텐서를 반환. 2차원 텐서의 dim=0은 각 분류(class)에 대한 원시(raw) 예측값 10개가, dim=1에는 각 출력의 개별 값들이 해당

In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


#### 모델 계층(Layer)
- 28x28 크기의 이미지 3개로 구성된 미니배치로 신경망 통과시 상황 확인

In [10]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


- nn.Flatten
	- 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환

In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


- nn.Linear
	- 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈

In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


- nn.ReLU
	- 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수

In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3817,  0.0150,  0.1008,  0.6268,  0.5137,  0.5808, -0.1526, -0.9321,
          0.3466, -0.0697,  0.1072,  0.5624, -0.2288,  0.3516, -0.0416,  0.1991,
         -0.2581,  0.0297, -0.0136, -0.0126],
        [-0.1173, -0.0389,  0.2802,  0.8919,  0.2533,  0.4601, -0.0183, -0.4285,
          0.2026,  0.1876,  0.2832,  0.5587, -0.0332,  0.1290,  0.1378,  0.3556,
         -0.1589,  0.4750,  0.5000,  0.2947],
        [ 0.0300, -0.0659,  0.1813,  0.8515,  0.3314,  0.3685,  0.1116, -0.3881,
         -0.0261,  0.3677,  0.0309,  0.3905, -0.2405,  0.3239, -0.0054,  0.1455,
         -0.4303,  0.2669, -0.0348,  0.1886]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3817, 0.0150, 0.1008, 0.6268, 0.5137, 0.5808, 0.0000, 0.0000, 0.3466,
         0.0000, 0.1072, 0.5624, 0.0000, 0.3516, 0.0000, 0.1991, 0.0000, 0.0297,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.2802, 0.8919, 0.2533, 0.4601, 0.0000, 0.0000, 0.2026,
         0.1876, 0.2832, 0.5587, 0.0000, 0.1290, 0.13

- nn.Sequential

In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

- nn.Softmax

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

#### 모델 매개변수

In [17]:
print(f"Model structure: {model}\n\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)




In [18]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0262,  0.0253,  0.0027,  ..., -0.0264, -0.0243, -0.0319],
        [-0.0054,  0.0059, -0.0174,  ...,  0.0209,  0.0073, -0.0345]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0205,  0.0306], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0331, -0.0377, -0.0209,  ..., -0.0313,  0.0074, -0.0311],
        [-0.0197, -0.0180, -0.0331,  ...,  0.0359, -0.0027,  0.0429]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([ 0.0386, -0.0423], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0281, -0.0099, -0.0277,  ..., -0.0331, -0.0406,  0.0350],
        [ 0.0129,  0.0407, -0.0261,